In [ ]:
import wikipedia
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import nltk
import csv
import pickle
import nltk
from nltk.stem import PorterStemmer

import re

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nour\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Collecting the data

In [ ]:
Training_Data_Animals = ''
Test__Data_Animals = ''
Training_Data_Arts = ''
Test__Data_Arts = ''

In [ ]:
duck = wikipedia.page("Ducks Animals")
cat = wikipedia.page("Cats Animals")
dog = wikipedia.page("Dogs Animals")
mouse = wikipedia.page("Mouse Animals")
turtle = wikipedia.page("Turtles Animals")
parrot = wikipedia.page("Parrot Animals")

In [ ]:
if (len(duck.content.split()) > 100):
    print("ok")
    Training_Data_Animals = Training_Data_Animals + duck.content

if (len(cat.content.split()) > 100):
    print("ok")
    Training_Data_Animals = Training_Data_Animals + cat.content

if (len(dog.content.split()) > 100):
    print("ok")
    Training_Data_Animals = Training_Data_Animals + dog.content

if (len(mouse.content.split()) > 100):
    print("ok")
    Training_Data_Animals = Training_Data_Animals + mouse.content

if (len(turtle.content.split()) > 100):
    print("ok")
    Training_Data_Animals = Training_Data_Animals + turtle.content

if (len(parrot.content.split()) > 100):
    print("ok")
    Test__Data_Animals = Test__Data_Animals + parrot.content

ok
ok
ok
ok
ok
ok


In [ ]:
Art1 = wikipedia.page("Ceramic art")
Art2 = wikipedia.page("Conceptual art")
Art3 = wikipedia.page("Drawing")
Art4 = wikipedia.page("Photography")
Art5 = wikipedia.page("Sculpture art")
Art6 = wikipedia.page("Performing arts")

In [ ]:
if (len(Art1.content.split()) > 100):
    print("ok")
    Training_Data_Arts = Training_Data_Arts + Art1.content

if (len(Art2.content.split()) > 100):
    print("ok")
    Training_Data_Arts = Training_Data_Arts + Art2.content

if (len(Art3.content.split()) > 100):
    print("ok")
    Training_Data_Arts = Training_Data_Arts + Art3.content

if (len(Art4.content.split()) > 100):
    print("ok")
    Training_Data_Arts = Training_Data_Arts + Art4.content

if (len(Art5.content.split()) > 100):
    print("ok")
    Training_Data_Arts = Training_Data_Arts + Art5.content

if (len(Art6.content.split()) > 100):
    print("ok")
    Test__Data_Arts = Test__Data_Arts + Art6.content

ok
ok
ok
ok
ok
ok


In [ ]:
Training_Data_Animals = re.split('[.]', Training_Data_Animals)
Training_Data_Arts = re.split('[.]', Training_Data_Arts)
Test__Data_Animals = re.split('[.]', Test__Data_Animals)
Test__Data_Arts = re.split('[.]', Test__Data_Arts)

In [ ]:
Train_X = Training_Data_Animals + Training_Data_Arts        
Test_X = Test__Data_Animals + Test__Data_Arts               


# animals ->  0        Arts ->  1

train_y = np.append(np.zeros(len(Training_Data_Animals)), np.ones(len(Training_Data_Arts)))
test_y = np.append(np.zeros(len(Test__Data_Animals)), np.ones(len(Test__Data_Arts)))

In [ ]:
header = ['sentences','lable' ]

with open('train.csv', 'w', encoding='UTF8', newline='') as f:

    writer = csv.writer(f)

    writer.writerow(header)

        
    for i in range (len (Train_X)):

        data = [Train_X[i],int(train_y[i])]


        writer.writerow(data)

In [ ]:
header = ['sentences','lable' ]

with open('test.csv', 'w', encoding='UTF8', newline='') as f:

    writer = csv.writer(f)

    writer.writerow(header)

        
    for i in range (len (Test_X)):

        data = [Test_X[i],int(test_y[i])]


        writer.writerow(data)

# 0. Import The data

In [ ]:
training_raw_Data = pd.read_csv("train.csv")
Test_raw_Data = pd.read_csv("test.csv")

training_raw_Data = training_raw_Data.dropna()
Test_raw_Data = Test_raw_Data.dropna()


# 1. The Data

## 1.1. get the data and the labels

In [ ]:
X_train = training_raw_Data["sentences"].to_numpy().reshape(-1,1)
X_test = Test_raw_Data["sentences"].to_numpy().reshape(-1,1)

y_train = training_raw_Data["lable"].to_numpy().reshape(-1,1)
y_test = Test_raw_Data["lable"].to_numpy().reshape(-1,1)

In [ ]:
# Check the shape

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2595, 1)
(294, 1)
(2595, 1)
(294, 1)


## 1.2 Preprocessing 

In [ ]:
def preprocessing (doc):

    # remove punctuation

    clean_doc = ""
    
    punctuation = "!#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789" 
    punctuation = punctuation + chr(34) + chr(39)


    
    for i in range (len(doc)):
        if not (doc[i] in punctuation):
            clean_doc = clean_doc + doc[i]


    # convert to lowercase

    clean_doc = clean_doc.lower()

    # Note: The rest of the preprocessing will be after tokenization step

    return clean_doc


## 1.3. Tokenization

In [ ]:
def get_tokens(doc):

    stemmer = PorterStemmer()

    tokens = []

    words = doc.split()



    for word in words:
        word = stemmer.stem(word)
        if not word in tokens:
            if not word in stop_words:
                tokens.append(word)

    return tokens
    

# 2. The Model

## 2.1. Calculte class prob

In [ ]:
def class_prob (doc_label):

    Animal_Count = 0
    Art_count = 0
    m = doc_label.shape[0]

    for i in range (m):
        if (doc_label[i][0] == 0):
            Animal_Count = Animal_Count + 1
        else:
            Art_count = Art_count + 1

    return (Animal_Count/m , Art_count/m)
    

## 2.2. Clculate conditional Prob

### 2.2.1. Build Word Dictionary *{("word",class):count}*

In [ ]:
def build_dict (X_train,y_train):

    word_dict = {}

    for i in range(X_train.shape[0]):
        
        sample = preprocessing(X_train[i][0])
        tokens = get_tokens(sample)

        for token in tokens:
            key = (token,y_train[i][0])
            
            if key in word_dict:
                word_dict[key] += 1
            else:
                word_dict[key] = 1

    return word_dict


In [ ]:
word_dict =build_dict (X_train,y_train)

### 2.2.2. Calculate every word prob 

In [ ]:
def conditional_prob (word_dict):

    word_prob_dict = {}

    Animal_count = 0
    Art_count = 0

    k = len(word_dict)

    for key, _ in word_dict.items():
        if (key[1] == 0):
            Animal_count = Animal_count + 1
        else:
            Art_count = Art_count + 1

    for key, values in word_dict.items():

        pair = (key[1],key[0])

        if (key[1] == 0):
            word_prob_dict[pair] = (values + 1) / (Animal_count + k)
        else:
            word_prob_dict[pair] = (values + 1) / (Art_count + k)

    return word_prob_dict

            

In [ ]:
word_prob_dict = conditional_prob (word_dict)

*saving on hard disk and import*

In [ ]:
with open('probs.pkl', 'wb') as fp:
    pickle.dump(word_prob_dict, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


## 2.3. predict 

In [ ]:
def predict (test_doc):

    with open('probs.pkl', 'rb') as fp:
        word_prob_dict = pickle.load(fp)

    prob_Animal_doc = 1
    prob_Art_doc = 1
    prob_animal_word = 1
    prob_Art_word = 1
    Animal_prop, Art_prop =class_prob (y_train)

    tokens = get_tokens(preprocessing(test_doc))

    for token in tokens:

        Animal_pair = (0,token)
        Art_pair = (1,token)

        if Animal_pair in word_prob_dict:
            prob_animal_word = word_prob_dict[Animal_pair]
            prob_Animal_doc = prob_Animal_doc * prob_animal_word

        elif Art_pair in word_prob_dict:
            prob_Art_word = word_prob_dict[Art_pair]
            prob_Art_doc = prob_Art_doc * prob_Art_word

    prob_Animal_given_doc = prob_Animal_doc * Animal_prop
    prob_Art_given_doc = prob_Art_doc * Art_prop

    ratio = prob_Animal_given_doc / prob_Art_given_doc

    if ratio > 1:
        y_hat = 0
    elif ratio < 1:
        y_hat = 1

    return y_hat


# 3. Evaluation

In [ ]:
def calculate_acc (x_test, y_test):

    number_of_samples = x_test.shape[0]
    correct = 0

    for i in range(number_of_samples):
        y_hat = predict (x_test[i][0])
        if (y_hat == y_test[i][0]):
            correct = correct + 1

    return correct/number_of_samples

In [ ]:
calculate_acc (X_test, y_test)

0.608843537414966

Note: I Think the accuracy in not good because the words in class 1 < words in class 2 after preprocessing